In [1]:
import csv
import os
import pandas as pd
import re
import nltk
import jsonlines
import json
import numpy as np
import hdbscan
from pprint import pprint
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime as dt, timedelta as td
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize 

from sklearn.cluster import KMeans, MeanShift, Birch, AffinityPropagation
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
from kneed import KneeLocator
import time
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from tqdm import tqdm
from wordcloud import WordCloud
from scipy.spatial.distance import cdist
from numpy import dot
from numpy.linalg import norm

In [2]:

lemmatizer = WordNetLemmatizer()

In [3]:
hyd_tweets = pd.read_pickle("Tweets_kolkata_labelled")
hyd_news = pd.read_pickle("News_kolkata_labelled")
full_news = pd.read_csv("./corona_articles/toi.csv")



### Tweets in news ?

In [19]:
for line in full_news.iterrows():
    headline = line[1]['Headline']
    if 'black' in headline and 'kolkata' in line[1]['URL'] and line[1]['Date']>='2021-04-11':
        print(line[1]['URL'],line[1]['Date'])

http://timesofindia.indiatimes.com//city/kolkata/oxygen-black-marketeer-in-police-net/articleshow/82507330.cms 2021-05-09
http://timesofindia.indiatimes.com//city/kolkata/kol-youth-in-hosp-with-suspected-black-fungus/articleshow/82810000.cms 2021-05-20
http://timesofindia.indiatimes.com//city/kolkata/state-records-5-new-black-fungus-cases/articleshow/83017570.cms 2021-05-27


In [13]:
def preprocessText(text, remove_less_than=5):
    txt = re.sub('@[A-Za-z0-9]+','',text)
    #remove links
    txt = re.sub('https://t.co/[A-Za-z0-9]+', '', txt)
    #remove hashtags
    txt = re.sub('#[A-Za-z0-9]+', '', txt) 
    #remove punctuation
    txt = re.sub('[,\.!-?]', '', txt)
    #remove numbers
    txt = re.sub('([0-9]+)', '', txt)
    txt = txt.lower()
            
    text = ' '.join([lemmatizer.lemmatize(word).strip() for word in txt.split() if len(word)>=remove_less_than and word not in STOPWORDS]) # removing stems, short words and stop words
    return text

In [10]:
news = (hyd_news["news"].tolist())
tweet = (hyd_tweets["tweets"].tolist())

In [43]:
cnt = 0
for cur in tweet:
    text = preprocessText(cur)
    ans = cur.split()
    words = ["shortage","less","stop","lack"]
    for x in ans:
        if "" in x.lower():
            print(cur)
            print("-------")
            cnt += 1
#         for l in words:
#             if l in x:
#                 print(cur)
#                 print("--------")
#                 cnt += 1
#                 break
print(cnt)

BUT CITIZEN WANTS TO KNOW WHY NO SUPPORT EXTENDED TO #Maharashtra &amp;  #WESTBENGAL IN FIGHTING #COVID_19 VIOLATING FEDERAL SYSTEM OF LAW WHEN RESPONSIBLITY OF ANY #PANDEMIC IS JOINT RESPONSIBLITY OF CENTRAL+STATE GOVT &amp; WHY @CMOMaharashtra COULD NOT PAY SALARY #COVID_19 WARRIORS? https://t.co/cZBIOwIVlb
-------
Govt to provide 90% of average salary as pension to families who lost earning member to COVID-19
https://t.co/wFWRCUsx4l

via NaMo App https://t.co/RxO8ADh0Ah
-------
Govt to provide 90% of average salary as pension to families who lost earning member to COVID-19.
https://t.co/mdlms0KqCi
-------
Govt to provide 90% of average salary as pension to families who lost earning member to COVID-19
https://t.co/Aqr0YI1PF5 via NaMo App https://t.co/ZSmrBoaGpk
-------
government employees are awaiting for this. salary shall be received at month end. private firm employees (like me) are will die. 
https://t.co/NIQNRR08W8
-------
Jaydev Unadkat will donate 10% of his IPL salary to sup

In [41]:
amd_tweets = pd.read_pickle("Tweets_ahmedabad_labelled")
amd_news = pd.read_pickle("News_ahmedabad_labelled")


In [10]:
news = (amd_news["news"].tolist())
tweet = (amd_tweets["tweets"].tolist())

In [12]:
STOPWORDS = nltk.corpus.stopwords.words('english')

STOPWORDS.append("covid")
STOPWORDS.append("india")
STOPWORDS.append("corona")
STOPWORDS.append("coronavirus")
STOPWORDS.append("covid19")
STOPWORDS.append("covid19india")
STOPWORDS.append("pandemic")
STOPWORDS.append("virus")
STOPWORDS.append("world")
STOPWORDS.append("state")
STOPWORDS.append("vaccine")
STOPWORDS.append("vaccination")
STOPWORDS.append("delhi")
STOPWORDS.append("india")

In [94]:
for cur in tweet:
    text = preprocessText(cur)
    ans = text.split()
    for x in ans:
        if "stop" in x:
            print(cur)
            print("--------")

@revathitweets When u are not imposing lockdown Then why you are stopping them?? Need answer sir why are they being stopped nd many can loose their lifes 🙏🙏  @TelanganaCMO @KTRTRS @TelanganaDGP @RGVzoomin #Ambulance #COVID19 #COVIDEmergency2021 #COVIDEmergency2021 #Telangana
--------
@revathitweets When u are not imposing lockdown Then why you are stopping them?? Need answer sir why are they being stopped nd many can loose their lifes 🙏🙏  @TelanganaCMO @KTRTRS @TelanganaDGP @RGVzoomin #Ambulance #COVID19 #COVIDEmergency2021 #COVIDEmergency2021 #Telangana
--------
Center gave a statement that "Any person can go anywhere to get treatment" then why @TelanganaCMO  government is stopping COVID-19 positive people from entering Telanganna ? Does advanced care only reserved for 1 state when we have a common capital? 
#CovidIndia 
#COVID19
--------
@WHO is the biggest idiot,culprit and the real villain behind all this chaos intially. Stopped believing them long back. https://t.co/LeRtmZw3Md
---

No Confirmed Bed-No Entry: Special Report On Police Stops Ambulances Coming From AP To TS..
watch here&gt;&gt;https://t.co/CLm99r4hf7
 
#NTVTelugu #NTVNews #ambulance #APTSBoarder #Coronavirus
--------
@IndiaToday @Ashi_IndiaToday Breaking- 'This Is Unconstitutional': Telangana High Court Slams State Government For Stopping Ambulances Carrying Covid-19 Patients At Borders
--------
@IndiaToday @Ashi_IndiaToday Breaking- 'This Is Unconstitutional': Telangana High Court Slams State Government For Stopping Ambulances Carrying Covid-19 Patients At Borders

https://t.co/7g9LOyFmsW
--------
Breaking- 'This Is Unconstitutional': Telangana High Court Slams State Government For Stopping Ambulances Carrying Covid-19 Patients At Borders

https://t.co/7g9LOyFmsW
--------
‘Stop misleading people, don’t create false panic’: BJP’s Nadda writes to Sonia Gandhi over COVID-19 criticism Times Now https://t.co/tWPmm36krI
--------
#Telangana High Court express disappointment over #COVID19 situation in state

However, the practice has been stopped since couple of days.
--------
#COVID19 surge: Northern Railways' Delhi Division stops sale of platform tickets with immediate effect
--------
#Swiggy and #Zomato services stopped in Hyderabad.
#TelanganaLockDown #swiggyindia #zomato #fooddeliveryservice #hyderbadcitypolice #hyderabad #tfcmediapvtltd #Coronavirus #fitness #healthylifestyle #Lockdown #Ecommerce #FoodDelivery #Telangana #TelanganaDGP #OnlineFoodDelivery https://t.co/tRpBLLIlMM
--------
If doctors really want every one to be cured ...
Then why they are stopping #anandayya #AyurvedicTreatment ?
Why they are against the #anandayya ?
Are doctors just want money from patients?

#medicalmafia #CovidIndia #CoronavirusPandemic # https://t.co/vANZrmI9CE
--------
@KTRTRS @TelanganaDGP @KTRTRS sir delivery boys are providing home delivery of food to many #COVID19 infected families. If these delivery boys are stopped from doing the job will Hyderabad city police will deliver food to these house

Updated #AIIMS Guidelines 
Hope the mad scramble for Remdesevir stops.
Timely Steroids &amp; Oxygen are gamechangers, even now.
@harishpemde
@ProfSomashekhar 
@naveenthacker 
@Maninde56414410 
@srinivasaiims 
@drgsrao 
@Covid19Critical 
@COVIDIndiaTrack 
#COVIDSecondWave 
#COVID19 https://t.co/nNsrmQYXkP
--------
China built a hospital in one week when the virus was at its peak there.
What has stopped the Modi govt from doing the same?

#WhoFailedIndia

https://t.co/d5UoEQous9 #WhoFailedIndia
--------
#Puducherry administration decides to impose a #lockdown from Friday (April 23) night till Monday (April 26) morning to “stop spread of #Coronavirus”. 
#StayHome
--------
Karnataka Government Stops Vaccination Drive Below 45 Years Due To Shortage

#Karnataka #covid19 #VaccineShortage  
https://t.co/Ll4isbv38q
--------
Corona booming does not seem to be stopping now
https://t.co/nIyTXNpPn6
#coronavirus #CoronaVirusUpdate #secondwave #2laccasesdaily #india @UnionHealthMinister @PMOIndia
---

Titanic is going to sink in an hour, but these orchestra artists did what they can do to reduce the panic in passengers. 
I believe #IPL2021 is also such a thing. It should NOT-stop, it is the best way to relax for after never ending bad days due pandemic. #COVID19 #Ipl #India https://t.co/nkixkJxaix
--------
@Vedmalik1 I am 100% with you on this sir.

All "yatras" should be stopped till the time situation caused by COVID19 pandemic comes under control.

God would be happy about sensible decision taken by the "devout" if the Yatras are called off.
--------
B.1.617 variant of Covid-19 feared to be contributing to a surge in coronavirus cases in India has been found in over a dozen countries 

The WHO recently listed B.1.617 — as a “variant of interest" but stopped short of declaring it as "variant of concern"
--------
@kothabutchikot2 @VSReddy_MP @ncbn So true, once YSRCP came to the power name the M.P who had grabbed all the Mines in and Around Vizag, for which contractor 500 CR has be

## NEWS

In [99]:
CLUSTER = 15
text = np.array(amd_news.loc[amd_news['labels'] == CLUSTER]['news'])

print(len(text))
print(text[6:9])

9
['AHMEDABAD: Western Railway has decided to reduce the frequency of various trains from the state and terminated several trains due to the Covid situation. The Sabarmati-Bhagat Ki Kothi special trains will now run only four days a week.The Sabarmati-Bhagat Ki Kothi No. 4820 has gone from a daily train to a biweekly train. It will leave now from Sabarmati on Tuesdays and Fridays and from Bhagat Ki Kothi on Wednesdays and Sundays. The Bandra-Ahmedabad-Bikaner Express will not be making its run from from Bikaner on May 17 and from Bandra on May 18.Officials said Train No. 4804, Sabarmati-Bhagat Ki Kothi which was also a daily train will leave Sabarmati on Wednesdays and Sundays and Train No. 4803 will leave Bhagat-Ki-Koti on Mondays and Thursdays.Railway officials that the Shriganganagar to Hujur Saheb Nanded train will also remain cancelled till further notice.'
 'The state health and family welfare department officials said that the cyclone aftermath might need more hands at hospitals

## Tweets

In [58]:
CLUSTER = 14
start = 0
end = 100
embed_cluster = np.array(amd_tweets.loc[amd_tweets['labels'] == CLUSTER]['embedding'].tolist())
text = np.array(amd_tweets.loc[amd_tweets['labels'] == CLUSTER]['tweets'])
cl = np.reshape(np.mean(embed_cluster,axis=0),(1,768))
sscore = (1-cdist(cl,embed_cluster,metric='cosine'))*(-1)
top_2_idx = np.argsort(sscore[0])[start:end]
#print(np.unique(text))
top_2_values = [(text[i],sscore[0][i]) for i in top_2_idx]
pprint(top_2_values)
print(len(text))

[('@INCGujarat\n'
  ' #Helpline9099902255\n'
  'Pls help.\n'
  'Patient Name:Bhadreshbhai\n'
  'Mo.: 8200811042\n'
  'Requirement: Bed in Covid-19 hospital\n'
  'Place: Visnagar\n'
  '@vijayrupanibjp @JayantiRavi @CollectorMeh @GujHFWDept @CMOGuj '
  '@MoHFW_GUJARAT @Nitinbhai_Patel @ikumarkanani',
  -0.9221064941228336),
 ('@prashantramwani @aravindchaudhri @aditiraval @kumarmanish9 @AmitHPanchal '
  '@bknasamachar @SP_Banaskantha @CMOGuj @CollectorBK @MadhishParikh Here are '
  'the url of websites \n'
  'Check availability of Hospital beds, oxygen ,\xa0\n'
  'https://t.co/7u7U6mIruX\xa0\n'
  'https://t.co/rrSLYCUTAb\n'
  'https://t.co/GI8kF7rXd1\n'
  'https://t.co/acFvHZYPKn\xa0\n'
  'https://t.co/SurIPHBbtS',
  -0.9168129354078886),
 ('@_Nilay9194 @MadhishParikh @KrunalShah225 @kumarmanish9 @aravindchaudhri '
  '@FalakThakkarr @SaralPatel @arjunmodhwadia Here are the url of websites \n'
  'Check availability of Hospital beds, oxygen ,\xa0\n'
  'https://t.co/7u7U6n02Tx\xa0\n'
  'htt